# python链接mysql数据库

In [21]:
import pymysql
import pickle

In [22]:
with open('F:/data6/final_result.pkl', 'rb') as f:
    final_result_list = pickle.load(f)

## 读入字典

In [23]:
with open('F:/data6/final_fea_to_score_dict.pkl', 'rb') as f:
    fea_to_score_dict = pickle.load(f)
score_list = []
label_list = []
#标签和分数列表
for key in fea_to_score_dict:
    score_list.append(fea_to_score_dict[key] * 10)
    label_list.append(key)
print(score_list)
print(label_list)

#将分数处理至于天猫类似的分数
new_score_list = [0,0,0,0,0,0,0,0]
i = 0
for w in score_list:
    if w >= 10:
        new_score_list[i] = (95 + (w - 10)/10 * 5)/2 *0.1
    else:
        new_score_list[i] = (90 + (10 - w)/10 * 5)/2 *0.1
    i += 1
print(new_score_list)

#计算全部的分数，加权平均
sum_score = 0
for w in new_score_list:
    sum_score += w
sum_label = 0
for w in new_score_list:
    sum_label += w/sum_score * w
print(sum_label)

[9.687494394847336, 16.666666666666668, 12.336538461538462, 9.597889771598808, 10.33534946236559, 12.333333333333334, 7.041048644338118, 6.375]
['电池', '态度', '包装', '质量', '价格', '描述', '物流', '电量']
[4.507812640128817, 4.916666666666667, 4.8084134615384615, 4.5100527557100305, 4.75838373655914, 4.808333333333334, 4.573973783891548, 4.590625]
4.688891911959535


## 添加新属性，全部

In [24]:
new_score_list.insert(0, sum_label)
label_list.insert(0, '全部')

In [25]:
round_score_list = []
for w in new_score_list:
    a = round(w,2)
    round_score_list.append(a)
print(round_score_list)

[4.69, 4.51, 4.92, 4.81, 4.51, 4.76, 4.81, 4.57, 4.59]


In [26]:
from collections import defaultdict
label_to_num_dict = defaultdict(int)
i = 0
for w in label_list:
    i += 1
    label_to_num_dict[w] = i
print(label_to_num_dict)

defaultdict(<class 'int'>, {'全部': 1, '电池': 2, '态度': 3, '包装': 4, '质量': 5, '价格': 6, '描述': 7, '物流': 8, '电量': 9})


## 链接mysql，存标签

In [27]:
db = pymysql.connect('localhost', 'root', '', 'goods', charset='utf8')
cursor = db.cursor()
finally_label_list = list()
i = 0
for w in label_list:
    finally_label_list.append(w + '(' + str(round_score_list[i]) + ')')
    i += 1
print(finally_label_list)
for w in finally_label_list:
    sql = '''INSERT INTO label(label, goods)
            VALUES ('%s', %d)'''%\
            (w, 2)
    cursor.execute(sql)
    db.commit()
db.close()

['全部(4.69)', '电池(4.51)', '态度(4.92)', '包装(4.81)', '质量(4.51)', '价格(4.76)', '描述(4.81)', '物流(4.57)', '电量(4.59)']


## 链接mysql 存评论和对应的标签编码

In [8]:
db = pymysql.connect('localhost','root','','goods',charset='utf8')
cursor = db.cursor()
for w in final_result_list:
    print(w[0])
    list1 = []
    for n in w[0]:
        list1.append(label_to_num_dict[n])
    list11 = [0,0,0,0,0,0,0,0,0]
    for m in list1:
        list11[m-1] = 1
    print(list1)
    print(list11)
    sql = '''INSERT INTO comment(content, goods, label1, label2, label3, label4, 
    label5, label6, label7, label8, label9)
    VALUES ('%s', %d, %d, %d, %d, %d, %d, %d, %d, %d, %d)'''%\
    (w[1], 2, 1, list11[1], list11[2], list11[3],\
     list11[4], list11[5], list11[6], list11[7], list11[8]) 
    
    cursor.execute(sql)
    db.commit()
db.close()

{'包装', '电池', '态度'}
[4, 2, 3]
[0, 1, 1, 1, 0, 0, 0, 0, 0]
set()
[]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
{'质量'}
[5]
[0, 0, 0, 0, 1, 0, 0, 0, 0]
{'质量', '包装'}
[5, 4]
[0, 0, 0, 1, 1, 0, 0, 0, 0]
set()
[]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
set()
[]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
{'电池'}
[2]
[0, 1, 0, 0, 0, 0, 0, 0, 0]
set()
[]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
{'质量', '电池'}
[5, 2]
[0, 1, 0, 0, 1, 0, 0, 0, 0]
set()
[]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
{'质量', '电池', '价格'}
[5, 2, 6]
[0, 1, 0, 0, 1, 1, 0, 0, 0]
{'电池'}
[2]
[0, 1, 0, 0, 0, 0, 0, 0, 0]
{'物流', '电池', '价格'}
[8, 2, 6]
[0, 1, 0, 0, 0, 1, 0, 1, 0]
{'电池', '描述'}
[2, 7]
[0, 1, 0, 0, 0, 0, 1, 0, 0]
{'电池'}
[2]
[0, 1, 0, 0, 0, 0, 0, 0, 0]
{'价格'}
[6]
[0, 0, 0, 0, 0, 1, 0, 0, 0]
{'物流'}
[8]
[0, 0, 0, 0, 0, 0, 0, 1, 0]
{'物流'}
[8]
[0, 0, 0, 0, 0, 0, 0, 1, 0]
{'电池'}
[2]
[0, 1, 0, 0, 0, 0, 0, 0, 0]
{'质量', '电池'}
[5, 2]
[0, 1, 0, 0, 1, 0, 0, 0, 0]
{'物流', '电池'}
[8, 2]
[0, 1, 0, 0, 0, 0, 0, 1, 0]
{'质量', '物流', '电池'}
[5, 8, 2]
[0, 1, 0, 0, 1, 0, 0, 1, 0]
{'电池'}
[2]
[0, 1, 0, 0, 0, 0, 0, 0,

{'电池'}
[2]
[0, 1, 0, 0, 0, 0, 0, 0, 0]
{'电池'}
[2]
[0, 1, 0, 0, 0, 0, 0, 0, 0]
{'质量', '电池'}
[5, 2]
[0, 1, 0, 0, 1, 0, 0, 0, 0]
{'价格'}
[6]
[0, 0, 0, 0, 0, 1, 0, 0, 0]
{'电量'}
[9]
[0, 0, 0, 0, 0, 0, 0, 0, 1]
set()
[]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
set()
[]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
{'电池'}
[2]
[0, 1, 0, 0, 0, 0, 0, 0, 0]
{'质量', '电池'}
[5, 2]
[0, 1, 0, 0, 1, 0, 0, 0, 0]
{'价格'}
[6]
[0, 0, 0, 0, 0, 1, 0, 0, 0]
{'质量'}
[5]
[0, 0, 0, 0, 1, 0, 0, 0, 0]
set()
[]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
set()
[]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
{'电池'}
[2]
[0, 1, 0, 0, 0, 0, 0, 0, 0]
{'质量'}
[5]
[0, 0, 0, 0, 1, 0, 0, 0, 0]
set()
[]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
{'电池'}
[2]
[0, 1, 0, 0, 0, 0, 0, 0, 0]
set()
[]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
{'电池'}
[2]
[0, 1, 0, 0, 0, 0, 0, 0, 0]
{'质量'}
[5]
[0, 0, 0, 0, 1, 0, 0, 0, 0]
{'包装', '电池'}
[4, 2]
[0, 1, 0, 1, 0, 0, 0, 0, 0]
{'电池'}
[2]
[0, 1, 0, 0, 0, 0, 0, 0, 0]
{'质量', '价格'}
[5, 6]
[0, 0, 0, 0, 1, 1, 0, 0, 0]
{'质量', '物流'}
[5, 8]
[0, 0, 0, 0, 1, 0, 0, 1, 0]
{'包装'}
[4]
[0, 0, 0, 1, 0, 0, 0

set()
[]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
set()
[]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
{'质量'}
[5]
[0, 0, 0, 0, 1, 0, 0, 0, 0]
{'电池'}
[2]
[0, 1, 0, 0, 0, 0, 0, 0, 0]
set()
[]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
set()
[]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
{'质量'}
[5]
[0, 0, 0, 0, 1, 0, 0, 0, 0]
set()
[]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
set()
[]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
{'物流'}
[8]
[0, 0, 0, 0, 0, 0, 0, 1, 0]
set()
[]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
set()
[]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
{'质量'}
[5]
[0, 0, 0, 0, 1, 0, 0, 0, 0]
set()
[]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
{'电池'}
[2]
[0, 1, 0, 0, 0, 0, 0, 0, 0]
{'质量', '电池'}
[5, 2]
[0, 1, 0, 0, 1, 0, 0, 0, 0]
set()
[]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
{'电池'}
[2]
[0, 1, 0, 0, 0, 0, 0, 0, 0]
set()
[]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
set()
[]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
{'描述'}
[7]
[0, 0, 0, 0, 0, 0, 1, 0, 0]
set()
[]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
set()
[]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
{'物流', '电池'}
[8, 2]
[0, 1, 0, 0, 0, 0, 0, 1, 0]
{'电池'}
[2]
[0, 1, 0, 0, 0, 0, 0, 0, 0]
set()
[]
[0, 0, 0, 0, 0, 0, 0, 0, 0